# <center>Assignment 6</center>

## <center>Regression Using ANN</center>

**Atharva Ramgirkar 19BCE0114**

### <center>Table of Contents</center>

* [Importing Libraries](#1.-Importing-Libraries)
* [Getting the Data](#2.-Getting-the-Data)
* [Looking at the Data](#3.-Looking-at-the-Data)
* [Handling Null Values](#4.-Handling-Null-Values)
    * [Administration Column](#4.1-Administration-Column)
    * [Marketing Spend Column](#4.2-Marketing-Spend-Column)
* [One Hot Encoding](#5.-One-Hot-Encoding)
    * [Column Transformer](#5.1-Column-Transformer)
* [Spliting Data into Features and Target](#6.-Spliting-Data-into-Features-and-Target)
* [Scaling the Data](#7.-Scaling-the-Data)
* [Train Test Split](#8.-Train-Test-Split)
* [Model Building](#9.-Model-Building)
    * [Model Initialization](#9.1-Model-Initialization)
    * [Adding Hidden Layers and Dropout Layers](#9.2-Adding-Hidden-Layers-and-Dropout-Layers)
    * [Adding Output Layer](#9.3-Adding-Output-Layer)
* [Compiling Model](#10.-Compiling-Model)
* [Training Model](#11.-Training-Model)
* [Model Prediction on Individual Observation](#12.-Model-Prediction-on-Individual-Observation)
* [Testing Model](#13.-Testing-Model)
* [RMSE Score](#14.-RMSE-Score)

### 1. Importing Libraries

[Back to Top](#Table-of-Contents)

In [1]:
import pandas as pd
import numpy as np

# To Initialize the model
from tensorflow.keras.models import Sequential 
# To Add layers
from tensorflow.keras.layers import Dense 
# To Add Droupout Layers
from tensorflow.keras.layers import Dropout

import tensorflow as tf
import keras

# Split the Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
import matplotlib.pyplot as plt
%matplotlib inline

# For Encoding 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# For Scaling 
from sklearn.preprocessing import StandardScaler

import math

### 2. Getting the Data

[Back to Top](#Table-of-Contents)

In [2]:
df = pd.read_csv("50_Startups.csv")

### 3. Looking at the Data

[Back to Top](#Table-of-Contents)

In [3]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [4]:
df.shape

(50, 5)

### 4. Handling Null Values 

[Back to Top](#Table-of-Contents)

In [5]:
df.isnull().sum()

R&D Spend          0
Administration     1
Marketing Spend    1
State              0
Profit             0
dtype: int64

In [6]:
df[df['Administration'].isnull()].index.tolist()

[10]

In [7]:
df[df['Marketing Spend'].isnull()].index.tolist()

[6]

#### 4.1 Administration Column

[Back to Top](#Table-of-Contents)

In [8]:
df.iloc[10,:]

R&D Spend          101913.08
Administration           NaN
Marketing Spend    229160.95
State                Florida
Profit             146121.95
Name: 10, dtype: object

In [9]:
df[df['State']=="Florida"]['Administration'].mean()

122513.88333333335

In [3]:
df.loc[10,('Administration')] = df[df['State']=="Florida"]['Administration'].mean()

In [11]:
df.loc[10,:]

R&D Spend              101913.08
Administration     122513.883333
Marketing Spend        229160.95
State                    Florida
Profit                 146121.95
Name: 10, dtype: object

In [12]:
df['Administration'][10]

122513.88333333335

In [13]:
df.isnull().sum()

R&D Spend          0
Administration     0
Marketing Spend    1
State              0
Profit             0
dtype: int64

#### 4.2 Marketing Spend Column

[Back to Top](#Table-of-Contents)

In [14]:
df.iloc[6,:]

R&D Spend           134615.46
Administration      147198.87
Marketing Spend           NaN
State              California
Profit              156122.51
Name: 6, dtype: object

In [15]:
df[df['State']=="California"]['Marketing Spend'].mean()

185967.43624999997

In [4]:
df.loc[6,('Marketing Spend')] = df[df['State']=="California"]['Marketing Spend'].mean()

In [17]:
df.isnull().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

### 5. One Hot Encoding

[Back to Top](#Table-of-Contents)

In [18]:
df.State.value_counts()

New York      17
California    17
Florida       16
Name: State, dtype: int64

#### 5.1 Column Transformer

[Back to Top](#Table-of-Contents)

In [5]:
ct = ColumnTransformer([("oh",OneHotEncoder(),[0])],
                        remainder = "passthrough")

In [6]:
df_states = pd.DataFrame(ct.fit_transform(df.State.to_numpy().reshape(-1,1)),columns=['NY','Californial','Florida'])

In [21]:
df.State.to_numpy()

array(['New York', 'California', 'Florida', 'New York', 'Florida',
       'New York', 'California', 'Florida', 'New York', 'California',
       'Florida', 'California', 'Florida', 'California', 'Florida',
       'New York', 'California', 'New York', 'Florida', 'New York',
       'California', 'New York', 'Florida', 'Florida', 'New York',
       'California', 'Florida', 'New York', 'Florida', 'New York',
       'Florida', 'New York', 'California', 'Florida', 'California',
       'New York', 'Florida', 'California', 'New York', 'California',
       'California', 'Florida', 'California', 'New York', 'California',
       'New York', 'Florida', 'California', 'New York', 'California'],
      dtype=object)

In [7]:
df = pd.concat([df_states,df],axis=1)
df

,NY,Californial,Florida,R&D Spend,Administration,Marketing Spend,State,Profit
0,0.0,0.0,1.0,165349.20,136897.800000,471784.10000,New York,192261.83
1,1.0,0.0,0.0,162597.70,151377.590000,443898.53000,California,191792.06
2,0.0,1.0,0.0,153441.51,101145.550000,407934.54000,Florida,191050.39
3,0.0,0.0,1.0,144372.41,118671.850000,383199.62000,New York,182901.99
4,0.0,1.0,0.0,142107.34,91391.770000,366168.42000,Florida,166187.94
5,0.0,0.0,1.0,131876.90,99814.710000,362861.36000,New York,156991.12
6,1.0,0.0,0.0,134615.46,147198.870000,185967.43625,California,156122.51
7,0.0,1.0,0.0,130298.13,145530.060000,323876.68000,Florida,155752.60
8,0.0,0.0,1.0,120542.52,148718.950000,311613.29000,New York,152211.77
9,1.0,0.0,0.0,123334.88,108679.170000,304981.62000,California,149759.96


In [8]:
df = df.drop('State',axis=1)

In [24]:
df.head()

,NY,Californial,Florida,R&D Spend,Administration,Marketing Spend,Profit
0,0.0,0.0,1.0,165349.20,136897.80,471784.10,192261.83
1,1.0,0.0,0.0,162597.70,151377.59,443898.53,191792.06
2,0.0,1.0,0.0,153441.51,101145.55,407934.54,191050.39
3,0.0,0.0,1.0,144372.41,118671.85,383199.62,182901.99
4,0.0,1.0,0.0,142107.34,91391.77,366168.42,166187.94


In [9]:
df_vals = df.values

### 6. Spliting Data into Features and Target

[Back to Top](#Table-of-Contents)

In [10]:
X = df_vals[:,0:6]
y = df_vals[:,6]

In [27]:
X[0]

array([0.000000e+00, 0.000000e+00, 1.000000e+00, 1.653492e+05,
       1.368978e+05, 4.717841e+05])

In [28]:
y[0]

192261.83

### 7. Scaling the Data

[Back to Top](#Table-of-Contents)

In [11]:
sc = StandardScaler()

In [12]:
X = sc.fit_transform(X)
y = sc.fit_transform(y.reshape(-1,1))

In [31]:
X[0]

array([-0.71774056, -0.68599434,  1.39326109,  2.01641149,  0.55299984,
        2.15371621])

In [32]:
y[0]

array([2.01120333])

### 8. Train Test Split

[Back to Top](#Table-of-Contents)

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.27,
                                                 random_state=114)

### 9. Model Building

[Back to Top](#Table-of-Contents)

#### 9.1 Model Initialization

In [95]:
# Feed Forward 
model = Sequential()

#### 9.2 Adding Hidden Layers and Dropout Layers

In [96]:
model.add(Dense(units=6,
                kernel_initializer="random_uniform",
                activation="relu"))

model.add(Dropout(0.2))

model.add(Dense(units=6,
                kernel_initializer="random_uniform",
                activation="relu"))

#### 9.3 Adding Output Layer

In [97]:
# output layer
model.add(Dense(units=1,
                kernel_initializer="random_uniform"))

### 10. Compiling Model

[Back to Top](#Table-of-Contents)

In [98]:
optimizer = keras.optimizers.Adam(lr=0.1)
model.compile(optimizer='adam',loss="mse",metrics='mse')

### 11. Training Model

[Back to Top](#Table-of-Contents)

In [99]:
history = model.fit(X_train,y_train,batch_size=5,epochs=1000,validation_data=(X_test,y_test))

Epoch 1/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.9601 - mse: 0.9601 - val_loss: 1.1028 - val_mse: 1.1028
Epoch 2/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.9595 - mse: 0.9595 - val_loss: 1.1020 - val_mse: 1.1020
Epoch 3/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.9594 - mse: 0.9594 - val_loss: 1.1008 - val_mse: 1.1008
Epoch 4/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.9584 - mse: 0.9584 - val_loss: 1.0997 - val_mse: 1.0997
Epoch 5/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.9578 - mse: 0.9578 - val_loss: 1.0979 - val_mse: 1.0979
Epoch 6/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.9566 - mse: 0.9566 - val_loss: 1.0956 - val_mse: 1.0956
Epoch 7/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.9555 - mse: 0.9555 - val_loss: 1.0927 - val_mse: 1.0927
Epoch 8/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.9521 - mse: 0.9

8/8 [==============================] - 0s 3ms/step - loss: 0.2146 - mse: 0.2146 - val_loss: 0.1707 - val_mse: 0.1707
Epoch 64/1000
8/8 [==============================] - ETA: 0s - loss: 0.0811 - mse: 0.081 - 0s 3ms/step - loss: 0.1472 - mse: 0.1472 - val_loss: 0.1666 - val_mse: 0.1666
Epoch 65/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.2227 - mse: 0.2227 - val_loss: 0.1657 - val_mse: 0.1657
Epoch 66/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1347 - mse: 0.1347 - val_loss: 0.1657 - val_mse: 0.1657
Epoch 67/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1852 - mse: 0.1852 - val_loss: 0.1654 - val_mse: 0.1654
Epoch 68/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1596 - mse: 0.1596 - val_loss: 0.1651 - val_mse: 0.1651
Epoch 69/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.2250 - mse: 0.2250 - val_loss: 0.1668 - val_mse: 0.1668
Epoch 70/1000
8/8 [==============================] - 0s 3ms

8/8 [==============================] - 0s 3ms/step - loss: 0.2303 - mse: 0.2303 - val_loss: 0.1525 - val_mse: 0.1525
Epoch 125/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1138 - mse: 0.1138 - val_loss: 0.1500 - val_mse: 0.1500
Epoch 126/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1167 - mse: 0.1167 - val_loss: 0.1477 - val_mse: 0.1477
Epoch 127/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1172 - mse: 0.1172 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 128/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1299 - mse: 0.1299 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 129/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1647 - mse: 0.1647 - val_loss: 0.1451 - val_mse: 0.1451
Epoch 130/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1428 - mse: 0.1428 - val_loss: 0.1426 - val_mse: 0.1426
Epoch 131/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.1072 - mse: 0.1

8/8 [==============================] - 0s 2ms/step - loss: 0.1332 - mse: 0.1332 - val_loss: 0.1440 - val_mse: 0.1440
Epoch 187/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1306 - mse: 0.1306 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 188/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1173 - mse: 0.1173 - val_loss: 0.1359 - val_mse: 0.1359
Epoch 189/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.2994 - mse: 0.2994 - val_loss: 0.1348 - val_mse: 0.1348
Epoch 190/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1093 - mse: 0.1093 - val_loss: 0.1362 - val_mse: 0.1362
Epoch 191/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0739 - mse: 0.0739 - val_loss: 0.1374 - val_mse: 0.1374
Epoch 192/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1324 - mse: 0.1324 - val_loss: 0.1415 - val_mse: 0.1415
Epoch 193/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1215 - mse: 0.1

8/8 [==============================] - 0s 3ms/step - loss: 0.0732 - mse: 0.0732 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 248/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1638 - mse: 0.1638 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 249/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1413 - mse: 0.1413 - val_loss: 0.1203 - val_mse: 0.1203
Epoch 250/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1083 - mse: 0.1083 - val_loss: 0.1191 - val_mse: 0.1191
Epoch 251/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1237 - mse: 0.1237 - val_loss: 0.1188 - val_mse: 0.1188
Epoch 252/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1083 - mse: 0.1083 - val_loss: 0.1183 - val_mse: 0.1183
Epoch 253/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1198 - mse: 0.1198 - val_loss: 0.1151 - val_mse: 0.1151
Epoch 254/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1065 - mse: 0.1

...

8/8 [==============================] - 0s 2ms/step - loss: 0.1048 - mse: 0.1048 - val_loss: 0.0800 - val_mse: 0.0800
Epoch 740/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1336 - mse: 0.1336 - val_loss: 0.0803 - val_mse: 0.0803
Epoch 741/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0848 - mse: 0.0848 - val_loss: 0.0794 - val_mse: 0.0794
Epoch 742/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0524 - mse: 0.0524 - val_loss: 0.0793 - val_mse: 0.0793
Epoch 743/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0645 - mse: 0.0645 - val_loss: 0.0794 - val_mse: 0.0794
Epoch 744/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0567 - mse: 0.0567 - val_loss: 0.0785 - val_mse: 0.0785
Epoch 745/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1150 - mse: 0.1150 - val_loss: 0.0764 - val_mse: 0.0764
Epoch 746/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0564 - mse: 0.0

8/8 [==============================] - 0s 3ms/step - loss: 0.0515 - mse: 0.0515 - val_loss: 0.0751 - val_mse: 0.0751
Epoch 802/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0774 - mse: 0.0774 - val_loss: 0.0756 - val_mse: 0.0756
Epoch 803/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1568 - mse: 0.1568 - val_loss: 0.0751 - val_mse: 0.0751
Epoch 804/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0964 - mse: 0.0964 - val_loss: 0.0754 - val_mse: 0.0754
Epoch 805/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0720 - mse: 0.0720 - val_loss: 0.0765 - val_mse: 0.0765
Epoch 806/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0423 - mse: 0.0423 - val_loss: 0.0766 - val_mse: 0.0766
Epoch 807/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0715 - mse: 0.0715 - val_loss: 0.0767 - val_mse: 0.0767
Epoch 808/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1201 - mse: 0.1

8/8 [==============================] - 0s 2ms/step - loss: 0.0415 - mse: 0.0415 - val_loss: 0.0806 - val_mse: 0.0806
Epoch 863/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0835 - mse: 0.0835 - val_loss: 0.0787 - val_mse: 0.0787
Epoch 864/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0601 - mse: 0.0601 - val_loss: 0.0781 - val_mse: 0.0781
Epoch 865/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1158 - mse: 0.1158 - val_loss: 0.0788 - val_mse: 0.0788
Epoch 866/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.1209 - mse: 0.1209 - val_loss: 0.0802 - val_mse: 0.0802
Epoch 867/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0608 - mse: 0.0608 - val_loss: 0.0802 - val_mse: 0.0802
Epoch 868/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0484 - mse: 0.0484 - val_loss: 0.0791 - val_mse: 0.0791
Epoch 869/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0628 - mse: 0.0

8/8 [==============================] - 0s 2ms/step - loss: 0.1210 - mse: 0.1210 - val_loss: 0.0771 - val_mse: 0.0771
Epoch 924/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0638 - mse: 0.0638 - val_loss: 0.0781 - val_mse: 0.0781
Epoch 925/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0867 - mse: 0.0867 - val_loss: 0.0799 - val_mse: 0.0799
Epoch 926/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0504 - mse: 0.0504 - val_loss: 0.0806 - val_mse: 0.0806
Epoch 927/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0569 - mse: 0.0569 - val_loss: 0.0797 - val_mse: 0.0797
Epoch 928/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0426 - mse: 0.0426 - val_loss: 0.0802 - val_mse: 0.0802
Epoch 929/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0924 - mse: 0.0924 - val_loss: 0.0788 - val_mse: 0.0788
Epoch 930/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0942 - mse: 0.0

8/8 [==============================] - 0s 3ms/step - loss: 0.1447 - mse: 0.1447 - val_loss: 0.0810 - val_mse: 0.0810
Epoch 986/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0472 - mse: 0.0472 - val_loss: 0.0816 - val_mse: 0.0816
Epoch 987/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0420 - mse: 0.0420 - val_loss: 0.0831 - val_mse: 0.0831
Epoch 988/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0848 - mse: 0.0848 - val_loss: 0.0835 - val_mse: 0.0835
Epoch 989/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0767 - mse: 0.0767 - val_loss: 0.0802 - val_mse: 0.0802
Epoch 990/1000
8/8 [==============================] - ETA: 0s - loss: 0.3430 - mse: 0.343 - 0s 3ms/step - loss: 0.0829 - mse: 0.0829 - val_loss: 0.0762 - val_mse: 0.0762
Epoch 991/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.0739 - val_mse: 0.0739
Epoch 992/1000
8/8 [==============================] -

### 12. Model Prediction on Individual Observation

[Back to Top](#Table-of-Contents)

In [101]:
model.predict(X[[0]]),model.predict(X[[1]])

(array([[1.7110282]], dtype=float32), array([[1.6637356]], dtype=float32))

In [102]:
sc.inverse_transform(model.predict(X[[0]]))

array([[180284.52]], dtype=float32)

In [103]:
y[[0]]

array([[2.01120333]])

In [104]:
X[0]

array([-0.71774056, -0.68599434,  1.39326109,  2.01641149,  0.55299984,
        2.15371621])

In [105]:
sc.inverse_transform(y[0])

array([192261.83])

### 13. Testing Model

[Back to Top](#Table-of-Contents)

In [106]:
model.predict(sc.inverse_transform(X_test))

array([[ 73391.664],
       [ 62318.695],
       [113067.08 ],
       [ 58832.008],
       [ 99049.27 ],
       [124724.03 ],
       [ 47660.227],
       [102452.734],
       [ 87329.164],
       [119600.81 ],
       [ 76001.37 ],
       [ 48203.68 ],
       [ 62621.227],
       [ 31481.99 ]], dtype=float32)

In [107]:
sc.inverse_transform(y_test)

array([[108733.99],
       [ 97483.56],
       [156991.12],
       [ 96712.8 ],
       [129917.04],
       [191050.39],
       [ 71498.49],
       [152211.77],
       [125370.37],
       [182901.99],
       [110352.25],
       [ 77798.83],
       [103282.38],
       [ 35673.41]])

### 14. RMSE Score

[Back to Top](#Table-of-Contents)

In [100]:
math.sqrt(pow(abs(model.predict(sc.inverse_transform(X_test))-sc.inverse_transform(y_test)),2).mean())

40906.757023740945